In [1]:
import numpy as np
import cv2
import imutils
from imutils.video import VideoStream
import time
import os

In [2]:
def anonymize_face_simple(image, factor=3.0):
#Increasing the factor will increase the blurriness
    (h,w) = image.shape[:2]
    kW = int(w/factor)
    kH = int(h/factor)
    
    if kW%2 == 0:
        kW -= 1
    
    if kH%2 == 0:
        kH -= 1
    
    return cv2.GaussianBlur(image, (kW,kH), 0)

In [3]:
def anonymize_face_pixelate(image, blocks=3):
    (h,w) = image.shape[:2]
    xSteps = np.linspace(0,w,blocks+1,dtype="int")
    ySteps = np.linspace(0,h,blocks+1,dtype="int")
    
    for i in range(1,len(ySteps)):
        for j in range(1,len(xSteps)):
            
            startX = xSteps[j-1]
            startY = ySteps[i-1]
            endX = xSteps[j]
            endY = ySteps[i]
            
            roi = image[startY:endY, startX: endX]
            (B,G,R) = [int(x) for x in cv2.mean(roi)[:3]]
            cv2.rectangle(image,(startX,startY), (endX,endY), (B,G,R),-1)

    return image


Anonymize faces by detecting faces using Haar Cascade Classifier

In [7]:
face_cascade = cv2.CascadeClassifier('model_data/haarcascade_frontalface_default.xml')

In [8]:
print("Starting video stream")
vs = VideoStream(src=0).start()
time.sleep(2.0)

while True:
    frame = vs.read()
    frame = imutils.resize(frame,width=400)
    
    (h,w) = frame.shape[:2]
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,1.3,5)
    
    for (x,y,w,h) in faces:
        face = frame[y:y+h, x:x+w]
        face = anonymize_face_pixelate(face)
        frame[y:y+h, x:x+w] = face
    
    cv2.imshow("Frame",frame)
    
    key = cv2.waitKey(1)&0xFF
    
    if key==ord("q"):
        break
cv2.destroyAllWindows()
vs.stop()
vs.stream.release()
print("Video stream stopped.")

Starting video stream
Video stream stopped.


Anonymize faces by detecting faces using Deep Neural Network

In [13]:
protoxt_path = r'model_data/deploy.prototxt'
caffemodel_path = r'model_data/weights.caffemodel'

model = cv2.dnn.readNetFromCaffe(protoxt_path,caffemodel_path)


In [14]:
print("Starting Video stream")
vs = VideoStream(src=0).start()
time.sleep(2.0)
while True:
    frame = vs.read()
    frame = imutils.resize(frame,width=400)

    (h,w) = frame.shape[:2]
    # (104.0,177.0,123.0) is the mean value for the ImageNet training set.
    blob = cv2.dnn.blobFromImage(cv2.resize(frame,(300,300)),1.0,(h,w),(104.0,177.0,123.0)) 
    model.setInput(blob)
    detections = model.forward()

    for i in range(0,detections.shape[2]):
        box = detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX, startY, endX, endY) = box.astype("int")
        face = frame[startY:endY,startX:endX]
        confidence = detections[0,0,i,2]
        if(confidence>0.5):
            face = anonymize_face_pixelate(face)
            frame[startY:endY,startX:endX] = face

    cv2.imshow("Frame",frame)

    key = cv2.waitKey(1)&0xFF

    if key==ord("q"):
        break

cv2.destroyAllWindows()
vs.stop()
vs.stream.release()
print("[INFO]Video stream stopped")

Starting Video stream
[INFO]Video stream stopped
